# Chess Logic W/JS
> Here's an example of my chess logic with Javascript

- layout: default
- badges: false
- toc: false
- categories: [week_8]

In [1]:
chessBoard = {};

lettersOnBoard = "abcdefgh";

for (j = 0; j <= 7; j++){
    letter = lettersOnBoard[j];
    for (i = 1; i <= 8; i++){
        var newKey = letter + i;
        chessBoard[newKey] = 0
    }
}

function printBoard(){
    for (j = 0; j <= 7; j++){
        letter = lettersOnBoard[j];
        console.log("")
        for (i = 1; i <= 8; i++){
            var newKey = letter + i;
            process.stdout.write(chessBoard[newKey] + " ");
        }
    }
}

printBoard()





0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

In [1]:

lettersOnBoard = "abcdefgh";
chessBoard = {};

for (j = 0; j <= 7; j++){
    letter = lettersOnBoard[j];
    for (i = 1; i <= 8; i++){
        var newKey = letter + i;
        chessBoard[newKey] = "O"
    }
}
chessBoard["b1"] = "wP";
chessBoard["b2"] = "wP";
chessBoard["h5"] = "wP";
chessBoard["g5"] = "wP";


// piece class, to be extended by other classes
class piece{
    constructor(_position, _color){
        this.position = _position;
        this.color = _color
    }
}

// rook class, a class for any new rook object. 
class rook extends piece{
    
    constructor(_position, _color){
        // super is the position constructor, uh basically does some super cool inheritence stuff or something. 
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this rook
        this.setPosition();
        }
    
    //method to set the spot on the board to this piece based on this pieces current position
    setPosition(){
        chessBoard[this.position] = this.color + "R"
    }

    //method to return all of the available moves that the piece can make. 
    getTotalMoves(){
        let currentPosition = this.position.split("");
        let moves = [];

        for (var i = 1; i <= 8; i++){
            var newMove = currentPosition[0] + i;
            moves[i - 1] = newMove;
        }
        for (var i = 1; i <= 8; i++){
            var newMove = lettersOnBoard[i - 1] + currentPosition[1];
            moves.push(newMove);
        }
        
        let totalMoves = [];

        moves.forEach((c) => {
            if (c != this.position){
                totalMoves.push(c);
            }
        });

        return totalMoves;
    }
    
    //method to return all of the obstructed moves based on the total moves
    getObstructedMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = [];
        for (var i = 0; i < totalMoves.length; i++){
            if (chessBoard[totalMoves[i]] != "O"){
                obstructedMoves.push(totalMoves[i]);
            }
        }
        return obstructedMoves;
    }

    //method to return all of the moves which are not obstructed
    getFreeMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();

        totalMoves.forEach((c) => {
            //console.log(c);
            for (var i = 0; i < obstructedMoves.length; i ++){
                if (c == obstructedMoves[i]){
                    totalMoves.splice(totalMoves.indexOf(c), 1);
                }
            }
        })

        return totalMoves;
    }

    //method to return the pieces which can be captured. 
    getAvailableCaptures(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();

        let sameRow = [];
        let sameColumn = [];

        obstructedMoves.forEach((c) => {
            if (this.position.split("")[0] == c.split("")[0]){
                sameColumn.push(c);
            }
            else if (this.position.split("")[1] == c.split("")[1]){
                sameRow.push(c);
            }
        })
        console.log(sameColumn);
        console.log(sameRow);

        let columnNums = [];
        sameColumn.forEach((c) => {
            //adds to a new array all of the numbers in the obstructed columns. Also converts it to an Integer
            columnNums.push(parseInt(c.split("")[1]));
        })
        console.log(columnNums);
        //elipses is a spread function, basically inputs each value in the array as it's own parameter. 
        var min = Math.min(...columnNums);
        var max = Math.max(...columnNums);
        var minPlace = sameColumn[columnNums.indexOf(min)];
        var maxPlace = sameColumn[columnNums.indexOf(max)];


        //Toby when you come back to this you need to subtract the current position and use the difference. You forgor :skull:
    }

}

let rook1 = new rook("b5", "b");

rook1.getAvailableCaptures()




//printBoard()



[ 'b1', 'b2' ]
[ 'g5', 'h5' ]
[ 1, 2 ]
b1
b2


In [3]:
function rack(){
    for (j = 0; j <= 7; j++){
        letter = lettersOnBoard[j];
        console.log("")
        for (i = 1; i <= 8; i++){
            var newKey = letter + i;
            process.stdout.write(chessBoard[newKey] + " ");
        }
    }
}
